# KLIMATA CLIMATE EXPOSURE SCORE CALCULATION

In [ ]:
import sqlite3
import pandas as pd
from sklearn.preprocessing import RobustScaler

# Connect to database
conn = sqlite3.connect("stand_datasets.db")

# Load standardized climate data
df = pd.read_sql("SELECT * FROM CLIMATE_STANDARDIZED", conn)

# -------------------------------
# Step 1 — Air Quality Index (AQI)
# -------------------------------
# Literature-based pollutant weights used earlier
aq_weights = {
    'co': 0.15,
    'so2': 0.15,
    'o3': 0.15,
    'pm10': 0.25,
    'pm25': 0.30
}

df['air_quality_score'] = (
    df['co']   * aq_weights['co']   +
    df['so2']  * aq_weights['so2']  +
    df['o3']   * aq_weights['o3']   +
    df['pm10'] * aq_weights['pm10'] +
    df['pm25'] * aq_weights['pm25']
)

# ------------------------------------------------------
# Step 2 — Combine AQI + Heat Index + PR into final score
# ------------------------------------------------------
# Literature-based climate weights we used before
climate_weights = {
    'air_quality_score': 0.50,
    'heat_index':        0.30,
    'pr':                0.20
}

df['climate_exposure_score_raw'] = (
    df['air_quality_score'] * climate_weights['air_quality_score'] +
    df['heat_index']        * climate_weights['heat_index'] +
    df['pr']                * climate_weights['pr']
)

# -------------------------------
# Step 3 — Normalize final result
# -------------------------------
scaler = RobustScaler()
df['climate_exposure_score'] = scaler.fit_transform(
    df[['climate_exposure_score_raw']]
)

# -------------------------------
# Step 4 — Save to SQLite
# -------------------------------
result = df[['barangay_name', 'climate_exposure_score']]
result.to_sql("CLIMATE_EXPOSURE_SCORE", conn, if_exists='replace', index=False)

conn.close()
